# DOLFINx meshes

DOLFINx provides a range of simple built-in meshes that are useful for quick testing and solving basic problems. You can find more information about the available mesh types, as well as tools for creating, refining, and marking meshes, in the [DOLFINx mesh documentation](https://docs.fenicsproject.org/dolfinx/v0.9.0/python/generated/dolfinx.mesh.html).

In this tutorial, we will cover the following:

- How to create basic meshes using DOLFINx  
- How to apply mesh transformations  
- How to use these meshes in a FESTIM simulation  
- How to define and label subdomains on DOLFINx meshes  

## Built-in DOLFINx meshes

The `create_unit_square` function in DOLFINx generates a structured, 2D mesh of the unit square, divided into a specified number of cells in the x and y directions. This is useful for simple test cases and provides a convenient starting point for setting up simulations.

In [1]:
from dolfinx.mesh import create_unit_square
from mpi4py import MPI

nx, ny = 10, 10  # Number of divisions in x and y directions
mesh = create_unit_square(MPI.COMM_WORLD, nx, ny)

In [2]:
from dolfinx import plot
import pyvista

pyvista.start_xvfb()
pyvista.set_jupyter_backend("html")


tdim = mesh.topology.dim

mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Similarly, the `create_unit_cube` function creates a 3D mesh of the unit cube, subdivided into a specified number of cells along the x, y, and z directions. It is useful for testing and developing 3D simulations.

In [3]:
from dolfinx.mesh import create_unit_cube
from mpi4py import MPI

nx, ny, nz = 10, 10, 10  # Number of divisions in x, y, and z directions
mesh = create_unit_cube(MPI.COMM_WORLD, nx, ny, nz)

In [4]:
tdim = mesh.topology.dim

mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_isometric()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

The `create_rectangle` function enables the definition of a 2D rectangular mesh over any axis-aligned domain by specifying custom corner coordinates, unlike `create_unit_square`, which has a fixed domain. This provides more flexibility for modelling geometries of arbitrary size.

In [5]:
from dolfinx.mesh import create_rectangle
from mpi4py import MPI

mesh = create_rectangle(MPI.COMM_WORLD, [[0, 0], [2, 1]], [20, 10])

In [6]:
tdim = mesh.topology.dim

mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_isometric()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

The type of mesh cells can also be specified:

In [7]:
from dolfinx.mesh import CellType

mesh = create_unit_square(MPI.COMM_WORLD, 10, 10, cell_type=CellType.quadrilateral)

## Mesh transformations

Sometimes it can be useful to apply some transformations to the coordinates of a mesh. Unit conversion, moving a mesh, rotating...

These operations can be done by modifying the ``.geometry.x`` attribute of the dolfinx mesh.

### Scaling

Scaling all dimensions of a mesh:

In [8]:
mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)

mesh.geometry.x[:] *= 1e-3

Scaling only the first dimension of a mesh:

In [9]:
mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)

mesh.geometry.x[:, 0] *= 0.5

In [10]:
tdim = mesh.topology.dim

mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_isometric()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

### Translating

In [11]:
mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)

mesh.geometry.x[:, 0] += 10

### Rotating

Rotating 90 degrees

In [12]:
mesh = create_rectangle(MPI.COMM_WORLD, [[0, 0], [2, 1]], [20, 10])

x_vals = mesh.geometry.x[:, 0].copy()

mesh.geometry.x[:, 0] = mesh.geometry.x[:, 1].copy()
mesh.geometry.x[:, 1] = x_vals

In [13]:
tdim = mesh.topology.dim

mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

# show orientation
plotter.show_axes()

if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Rotate by 30 degrees:

In [14]:
from scipy.spatial.transform import Rotation

mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)

degrees = 30
rotation = Rotation.from_euler("z", degrees, degrees=True)
mesh.geometry.x[:,:] = rotation.apply(mesh.geometry.x)

In [15]:
tdim = mesh.topology.dim

mesh.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(mesh, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

# show orientation
plotter.show_axes()

if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("mesh.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Passing a dolfinx mesh to FESTIM

To pass a dolfinx mesh to a FESTIM model, simply pass it to a `festim.Mesh` object:

In [16]:
import festim as F

fenics_mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)

festim_mesh = F.Mesh(fenics_mesh)

## Defining subdomains

Now that we know how to pass a dolfinx mesh to FESTIM, we need to mark mesh entities to define subdomains (volume subdomains or surface subdomains).

```{note}
This will be greatly simplified once [PR #1005](https://github.com/festim-dev/FESTIM/pull/1005) is merged.
```

### Surface subdomains

To define surface subdomains, we make use of the ``festim.SurfaceSubdomain`` class and override the ``locate_boundary_facet_indices()`` method:

In [17]:
from dolfinx.mesh import locate_entities_boundary
import numpy as np


class TopSurface(F.SurfaceSubdomain):
    def locate_boundary_facet_indices(self, mesh):
        fdim = mesh.topology.dim - 1

        locator = lambda x: np.isclose(x[1], 1.0)

        indices = locate_entities_boundary(mesh, fdim, locator)
        return indices


class BottomSurface(F.SurfaceSubdomain):
    def locate_boundary_facet_indices(self, mesh):
        fdim = mesh.topology.dim - 1

        locator = lambda x: np.isclose(x[1], 0.0)

        indices = locate_entities_boundary(mesh, fdim, locator)
        return indices


top_surface = TopSurface(id=1)
bottom_surface = BottomSurface(id=2)

### Volume subdomains

Similarily for volume subdomains, we override the ``locate_subdomain_entities()`` method:

In [18]:
from dolfinx.mesh import locate_entities

# volume subdomains need a material
material = F.Material(name="test_material", D_0=1, E_D=0)


class TopVolume(F.VolumeSubdomain):
    def locate_subdomain_entities(self, mesh):
        locator = lambda x: x[1] >= 0.5
        entities = locate_entities(mesh, mesh.topology.dim, locator)
        return entities


class BottomVolume(F.VolumeSubdomain):
    def locate_subdomain_entities(self, mesh):
        locator = lambda x: x[1] <= 0.5
        entities = locate_entities(mesh, mesh.topology.dim, locator)
        return entities


top_volume = TopVolume(id=1, material=material)
bottom_volume = BottomVolume(id=2, material=material)

### Visualise subdomains

Subdomains can be visualised by passing them to a FESTIM problem and then call ``define_meshtags_and_measures()``. This will generate ``dolfinx.mesh.MeshTags`` objects that can then be used for visualisation:

In [19]:
my_model = F.HydrogenTransportProblem()
my_model.mesh = festim_mesh
my_model.subdomains = [top_surface, bottom_surface, top_volume, bottom_volume]

my_model.define_meshtags_and_measures()

print(my_model.facet_meshtags)
print(my_model.volume_meshtags)

``pyvista`` can be used for visualisation:

In [20]:
fdim = my_model.mesh.mesh.topology.dim - 1
tdim = my_model.mesh.mesh.topology.dim
my_model.mesh.mesh.topology.create_connectivity(fdim, tdim)
topology, cell_types, x = plot.vtk_mesh(
    my_model.mesh.mesh, tdim, my_model.volume_meshtags.indices
)

p = pyvista.Plotter(window_size=[800, 800])
grid = pyvista.UnstructuredGrid(topology, cell_types, x)
grid.cell_data["Cell Marker"] = my_model.volume_meshtags.values
grid.set_active_scalars("Cell Marker")
p.add_mesh(grid, show_edges=True)
if pyvista.OFF_SCREEN:
    figure = p.screenshot("volume_marker.png")
p.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [21]:
my_model.mesh.mesh.topology.create_connectivity(fdim, tdim)
topology, cell_types, x = plot.vtk_mesh(
    my_model.mesh.mesh, fdim, my_model.facet_meshtags.indices
)

p = pyvista.Plotter(window_size=[800, 800])
grid = pyvista.UnstructuredGrid(topology, cell_types, x)
grid.cell_data["Facet Marker"] = my_model.facet_meshtags.values
grid.set_active_scalars("Facet Marker")
p.add_mesh(grid, show_edges=True)
if pyvista.OFF_SCREEN:
    figure = p.screenshot("facet_marker.png")
p.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

The meshtags can also be stored to a VTX file using ``adios4dolfinx`` to be read later by another script:

In [22]:
import adios4dolfinx

my_model.facet_meshtags.name = "facet_tags"

# write
adios4dolfinx.write_meshtags("facet_tags.bp", my_model.mesh.mesh, my_model.facet_meshtags)

# read
ft = adios4dolfinx.read_meshtags("facet_tags.bp", my_model.mesh.mesh, meshtag_name="facet_tags")

## Complete example

### Implementation

In [23]:
import festim as F
from dolfinx.mesh import locate_entities_boundary
from dolfinx.mesh import locate_entities

import numpy as np

fenics_mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)

festim_mesh = F.Mesh(fenics_mesh)


class TopSurface(F.SurfaceSubdomain):
    def locate_boundary_facet_indices(self, mesh):
        fdim = mesh.topology.dim - 1
        indices = locate_entities_boundary(mesh, fdim, lambda x: np.isclose(x[1], 1.0))
        return indices


class BottomSurface(F.SurfaceSubdomain):
    def locate_boundary_facet_indices(self, mesh):
        fdim = mesh.topology.dim - 1
        indices = locate_entities_boundary(mesh, fdim, lambda x: np.isclose(x[1], 0.0))
        return indices


class TopVolume(F.VolumeSubdomain):
    def locate_subdomain_entities(self, mesh):
        entities = locate_entities(
            mesh,
            mesh.topology.dim,
            lambda x: x[1] >= 0.5,
        )
        return entities


class BottomVolume(F.VolumeSubdomain):
    def locate_subdomain_entities(self, mesh):
        entities = locate_entities(
            mesh,
            mesh.topology.dim,
            lambda x: x[1] <= 0.5,
        )
        return entities


material_top = F.Material(D_0=1, E_D=0)
material_bot = F.Material(D_0=2, E_D=0)


top_volume = TopVolume(id=1, material=material_top)
bottom_volume = BottomVolume(id=2, material=material_bot)

top_surface = TopSurface(id=1)
bottom_surface = BottomSurface(id=2)

my_model = F.HydrogenTransportProblem()
my_model.mesh = festim_mesh
my_model.subdomains = [top_surface, bottom_surface, top_volume, bottom_volume]

H = F.Species("H")
my_model.species = [H]

my_model.temperature = 400

my_model.boundary_conditions = [
    F.FixedConcentrationBC(subdomain=top_surface, value=1.0, species=H),
    F.FixedConcentrationBC(subdomain=bottom_surface, value=0.0, species=H),
]

my_model.settings = F.Settings(atol=1e-10, rtol=1e-10, transient=False)

my_model.initialise()
my_model.run()

### Visualisation

In [24]:
hydrogen_concentration = H.solution

topology, cell_types, geometry = plot.vtk_mesh(hydrogen_concentration.function_space)
u_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
u_grid.point_data["c"] = hydrogen_concentration.x.array.real
u_grid.set_active_scalars("c")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()

if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    figure = u_plotter.screenshot("concentration.png")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…